In [5]:
import spotipy
import base64
import requests
from spotipy.oauth2 import SpotifyClientCredentials
from creds import CLIENT_ID, CLIENT_SECRET
import pandas as pd
import time

In [6]:
# token
auth_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)


In [7]:
#Get spotify playlist ids and names

def get_all_playlists(user_id): #api call
    playlist_ids = []
    playlist_names = []
    playlists = sp.user_playlists(user_id) #api call
    #my_playlists = sp.user_playlist('z8ctw1rqti6nguka70zk1ishf')

    while playlists: # example provided in spotipy documentation
        for i, playlist in enumerate(playlists['items']):
            #print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
            playlist_ids.append(playlist['uri'][-22:])
            playlist_names.append(playlist['name'])
        if playlists['next']:
            playlists = sp.next(playlists)
            
        else:
            playlists = None
    return playlist_ids, playlist_names


In [8]:
# Get track info from playlists into a dataframe - separated out API calls

def get_playlist_tracks(playlist_id):#api call
    try: 
        tracks = sp.playlist_tracks(playlist_id, limit = 100, fields='items(track(id, name, artists, album(id, name)))')
        return tracks['items']
    
    except Exception as e:
        print(f"Error fetching {e}")
        return []

def get_track_details(track_id):
    try:
        track_info = sp.track(track_id)
        return track_info['popularity']
    except Exception as e:
        print(f"Error fetching track {e}")
        return None

def get_audio_features(track_id): #api call
    try:
        audio_stuffs = sp.audio_features(track_id)[0] if track_id else None 
        return audio_stuffs
    except Exception as e:
        print(f"Error fetching {e}")
        return None
    
def get_tracks(playlist_id, playlist_name):
    music_data = []
    tracks =  get_playlist_tracks(playlist_id)  

    for track_info in tracks['items']:
        track = track_info['track']

        if track:
            track = track_info['track']
            track_name = track['name']
            artists = ', '.join([artist['name'] for artist in track['artists']])
            album_name = track['album']['name']
            album_id = track['album']['id']
            track_id = track['id']
        
        audio_stuffs = get_audio_features(track_id) #api call
        popularity = get_track_details(track_id) #api call


        track_data = {
            'Playlist Name': playlist_name,
            'Track': track,
            'Track Name': track_name,
            'Artist': artists,
            'Album Name': album_name,
            'Playlist ID': playlist_id,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Acousticness': audio_stuffs['acousticness'] if audio_stuffs else None,
            'Danceability': audio_stuffs['danceability'] if audio_stuffs else None,
            'Energy': audio_stuffs['energy'] if audio_stuffs else None,
            'Instrumentalness': audio_stuffs['instrumentalness'] if audio_stuffs else None,
            'Loudness': audio_stuffs['loudness'] if audio_stuffs else None,
            'Speechiness': audio_stuffs['speechiness'] if audio_stuffs else None,
            'Tempo': audio_stuffs['tempo'] if audio_stuffs else None,
            'Mode': audio_stuffs['mode'] if audio_stuffs else None,
            'Valence': audio_stuffs['valence'] if audio_stuffs else None
        }

        music_data.append(track_data)
    return music_data

user_id = 'spotify'
playlist_ids, playlist_names = get_all_playlists(user_id)

all_tracks = []       
for playlist_id, playlist_name in zip(playlist_names, playlist_ids): #Test with first 10 IDs
    #print(f"Getting tracks from playlist ID: {id}")
    tracks = get_tracks(playlist_id,playlist_names) #Two API calls per function call
    all_tracks.extend(tracks)
    time.sleep(5) # to keep the spotify api server from giving me a 429 error when they finally take me back

#Test with one playlist id
# playlist_id = playlist_ids[0]
# track = get_tracks(playlist_id)
# all_tracks.extend(track)

spotify_track_df = pd.DataFrame(all_tracks, columns=['Playlist Name','Track Name', 'Artist', 'Album Name', 'Playlist ID','Album ID', 'Track ID',
                                                     'Acousticness', 'Danceability','Energy', 'Instrumentalness', 
                                                     'Loudness', 'Speechiness', 'Tempo', 'Mode', 'Valence'])

spotify_track_df.head()


Error fetching http status: 400, code:-1 - Unsupported URL / URI., reason: None


TypeError: list indices must be integers or slices, not str